In [1]:
import os
import numpy as np
import pandas as pd
from scipy.sparse import load_npz
from src.utils import topic_clustering as TopicClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

In [2]:
data = pd.read_parquet("data/articles_processed.parquet")
tf    = load_npz('models/tf_unigram_articles.npz')
tfidf = load_npz('models/tfidf_unigram_articles.npz')
vocabulary = pd.read_csv("models/vocabulary_unigram_articles.csv").ngrams
doc_top_wals = np.load(f'models/doc_topics_wals_unigram_articles.npy').astype('float32')
word_top_wals = np.load(f'models/word_topics_wals_unigram_articles.npy').astype('float32')

assert tf.shape == tfidf.shape, 'Rever BOW'
assert tf.shape[0] == data.shape[0], 'Rever BOW'
assert tf.shape[0] == doc_top_wals.shape[0], 'Rever Modelagem de Tópicos'
assert word_top_wals.shape[1] == doc_top_wals.shape[1], 'Rever Modelagem de Tópicos'

**Verificando valores faltantes em todas as features**

In [113]:
data_raw = pd.read_parquet("data/articles.parquet")

In [114]:
data_raw

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,None,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,None,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,None,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,None,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,None,http://www1.folha.uol.com.br/mercado/2017/10/1...
...,...,...,...,...,...,...
167048,"Em cenário de crise, tucano Beto Richa assume ...",O tucano Beto Richa tinha tudo para começar se...,2015-01-01,poder,None,http://www1.folha.uol.com.br/poder/2015/01/156...
167049,Filho supera senador Renan Calheiros e assume ...,O economista Renan Filho (PMDB) assume nesta q...,2015-01-01,poder,None,http://www1.folha.uol.com.br/poder/2015/01/156...
167050,"Hoje na TV: Tottenham x Chelsea, Campeonato In...",Destaques da programação desta quinta-feira (1...,2015-01-01,esporte,None,http://www1.folha.uol.com.br/esporte/2015/01/1...
167051,Kim Jong-un diz estar aberto a se reunir com p...,"O líder norte-coreano, Kim Jong-un, disse nest...",2015-01-01,mundo,None,http://www1.folha.uol.com.br/mundo/2015/01/156...


In [112]:
data

,title,text,date,category,subcategory,link,cluster_wals_200,cluster_wals_50
0,lula diz esta lascado que ainda forca cabo ele...,com possibilidade uma condenacao impedir candi...,2017-09-10,poder,None,http://www1.folha.uol.com.br/poder/2017/10/192...,85,13
1,decidi escrava mulheres sofrem diz cantora oum...,para oumou sangare cantora ativista malinesa s...,2017-09-10,ilustrada,None,http://www1.folha.uol.com.br/ilustrada/2017/10...,102,43
2,tres reportagens folha ganham premio petrobras...,tres reportagens folha vencedoras premio petro...,2017-09-10,poder,None,http://www1.folha.uol.com.br/poder/2017/10/192...,131,41
3,filme star wars os ultimos jedi ganha trailer ...,a disney divulgou noite desta segundafeira no...,2017-09-10,ilustrada,None,http://www1.folha.uol.com.br/ilustrada/2017/10...,154,25
4,cbss inicia acordos fintechs quer credito dig...,o cbss banco holding elopar socios bradesco ba...,2017-09-10,mercado,None,http://www1.folha.uol.com.br/mercado/2017/10/1...,109,39
...,...,...,...,...,...,...,...,...
167048,em cenario crise tucano beto richa assume segu...,o tucano beto richa tudo comecar ano forma con...,2015-01-01,poder,None,http://www1.folha.uol.com.br/poder/2015/01/156...,74,13
167049,filho supera senador renan calheiros assume ne...,o economista renan filho pmdb assume nesta qui...,2015-01-01,poder,None,http://www1.folha.uol.com.br/poder/2015/01/156...,63,13
167050,hoje tv tottenham x chelsea campeonato ingles ...,destaques programacao desta quintafeira o h ...,2015-01-01,esporte,None,http://www1.folha.uol.com.br/esporte/2015/01/1...,191,6
167051,kim jongun diz estar aberto se reunir presiden...,o lider nortecoreano kim jongun disse nesta qu...,2015-01-01,mundo,None,http://www1.folha.uol.com.br/mundo/2015/01/156...,179,1


In [5]:
data.isnull().mean()[data.isnull().any()].rename('Taxa de Valores Faltantes')

subcategory    0.822601
Name: Taxa de Valores Faltantes, dtype: float64

**Verificando strings vazias: tipo de valor faltante**

In [6]:
(data[['title', 'text', 'category', 'link']].applymap(lambda x: len(x)) == 0).sum()

title         3
text        766
category      0
link          0
dtype: int64

In [7]:
data.category.value_counts()

poder                           22022
colunas                         21622
mercado                         20970
esporte                         19730
mundo                           17130
cotidiano                       16967
ilustrada                       16345
opiniao                          4525
paineldoleitor                   4011
saopaulo                         3955
tec                              2260
tv                               2142
educacao                         2118
turismo                          1903
ilustrissima                     1411
ciencia                          1335
equilibrioesaude                 1312
sobretudo                        1057
bbc                               980
folhinha                          876
empreendedorsocial                841
comida                            828
asmais                            548
ambiente                          491
seminariosfolha                   379
serafina                          334
o-melhor-de-

## Verificando BOW

In [8]:
words_df = pd.DataFrame({"word":vocabulary.values, 'freq': np.asarray(tf.sum(axis=0))[0]}
                       ).sort_values('freq', ascending=False)

**Palavras Mais Comuns**

In [9]:
words_df.head(20).T

,4050,22800,47708,61435,33597,61982,24268,2331,4025,9741,54102,43052,23138,46871,49744,50699,50014,64303,47467,14347
word,anos,disse,no,sao,governo,segundo,ele,ainda,ano,brasil,presidente,mais,diz,na,pais,paulo,para,sua,nesta,com
freq,148358,138855,138844,138626,131625,128596,125899,125491,124475,120982,118032,117031,116539,112803,111588,109385,105829,93992,91116,88436


In [10]:
' '.join(words_df.iloc[:250].word)

'anos disse no sao governo segundo ele ainda ano brasil presidente mais diz na pais paulo para sua nesta com pode pessoas contra seu dois mas apos dia isso foi da milhoes afirmou do rio desde tem fazer maior parte tres caso casa outros por mil menos ser mundo apenas como federal acordo estado porque ja todos hoje alem empresa tempo onde antes dilma vez folha agora ela grande eu ha tambem primeiro cidade deve neste esta publico dias politica nos mesmo outro grupo duas nova durante seus bem primeira nacional afirma empresas semana novo passado cada temer trabalho assim ao meio vida mercado ate bilhoes ministro melhor policia fim processo justica cerca conta estados eles todo quem quando forma brasileiro final ministerio ter depois sido us outras sempre quatro meses eua decisao relacao podem crise alguns momento camara fez regiao entao muito dinheiro faz so dos lado brasileira segunda projeto desta qualquer enquanto quanto paises pouco exemplo historia quase partir lula valor saude numero

# Tópicos

In [11]:
%%time
topwords_relevance_df_wals_topic, top5_words_relevance_wals_topic = TopicClustering.view_topics(word_top_wals, vocabulary, relevance=True, n_top_words=50)
topwords_weight_df_wals_topic, top5_words_weight_wals_topic       = TopicClustering.view_topics(word_top_wals, vocabulary, relevance=False, n_top_words=50)

CPU times: user 2.96 s, sys: 36 ms, total: 2.99 s
Wall time: 2.99 s


In [12]:
top5_words_relevance_wals_topic

0              dados | queda | afeta | coordenacao | sua
1               fato | entre | menos | parece | policial
2      diretor | uruguai | argentina | paraguai | pes...
3                  opiniao | militar | bola | vezes | pm
4        negociacoes | acordo | plano | trump | proposta
                             ...                        
295             senador | aecio | neves | senado | vezes
296    pontos | estados | americanos | unidos | ameri...
297             equipe | titulo | com | vitoria | branco
298      criancas | publico | crianca | filhos | familia
299     pesquisa | datafolha | ate | alckmin | pesquisas
Length: 300, dtype: object

In [13]:
df_words5_topics = pd.DataFrame({"topic_words_weight": top5_words_weight_wals_topic,"topic_words_relevance": top5_words_relevance_wals_topic})

In [14]:
df_words5_topics.iloc[0:60]

,topic_words_weight,topic_words_relevance
0,sua | dados | tambem | queda | reportagem,dados | queda | afeta | coordenacao | sua
1,fato | menos | fim | entre | janeiro,fato | entre | menos | parece | policial
2,diretor | foi | para | dois | paulo,diretor | uruguai | argentina | paraguai | pes...
3,militar | opiniao | vezes | politica | grande,opiniao | militar | bola | vezes | pm
4,acordo | plano | bilhoes | proposta | seu,negociacoes | acordo | plano | trump | proposta
5,foram | defesa | dias | cinco | joao,defesa | foram | ensino | prova | enem
6,primeiro | servicos | poder | ministerio | rec...,servicos | poder | primeiro | recursos | postos
7,camara | cunha | deputados | senado | eduardo,cunha | camara | deputados | plenario | pmdbrj
8,deputados | camara | deputado | caso | federal,deputados | deputado | camara | denuncia | par...
9,quartafeira | europa | mundo | fiscal | impostos,europa | impostos | fiscal | quartafeira | adm...


# Clusters

In [3]:
clustering_outputs_wals_200  = np.load('models/clusters_wals_unigram_articles_200_outputs.npz')
data['cluster_wals_200'] = clustering_outputs_wals_200['index']

In [4]:
%%time
word_cluster_wals_200 = TopicClustering.get_word_cluster_matrix(data.cluster_wals_200, tf).astype('float32')
topwords_relevance_df_wals_200, top5_words_relevance_wals_200 = TopicClustering.view_topics(word_cluster_wals_200, vocabulary, relevance=True, n_top_words=50)
topwords_weight_df_wals_200, top5_words_weight_wals_200       = TopicClustering.view_topics(word_cluster_wals_200, vocabulary, relevance=False, n_top_words=50)

CPU times: user 2.25 s, sys: 61 ms, total: 2.31 s
Wall time: 2.31 s


In [5]:
df_words5_clusters = pd.DataFrame({'n_instances': data.cluster_wals_200.value_counts(),
                                   "cluster_words_weight": top5_words_weight_wals_200, 
                                   "cluster_words_relevance": top5_words_relevance_wals_200})

In [35]:
df_words5_clusters.to_csv("outputs/faltando.csv", index=False)

In [21]:
clusters_sample = [60,84,92,45,168,50,71,113,87,140]

In [6]:
topwords_relevance_df_wals_200.iloc[:10,:7]

,0,1,2,3,4,5,6
0,cunha,sp,joesley,indice,indigenas,inbev,colunaobituariogrupofolhacombr
1,cassacao,leitorgrupofolhacombr,jbs,yuan,funai,sabmiller,anuncios
2,etica,paulo,loures,xangai,indios,ab,missas
3,pinato,sao,batista,china,terras,yahoo,mortes
4,camara,leitores,saud,fed,desmatamento,kroton,netos
5,conselho,enviando,janot,shenzhen,indigena,estacio,veja
6,peemedebista,envie,temer,bce,amazonia,us,gostava
7,deputado,colunistas,pgr,csi,renca,buffett,morreu
8,maranhao,acontecimentos,miller,ssec,demarcacao,transacao,filhos
9,pmdbrj,relevantes,delacao,pmi,hectares,aquisicao,sobrinhos


In [31]:
df_words5_clusters.iloc[:7, 1:]

,cluster_words_weight,cluster_words_relevance
0,cunha | camara | presidente | conselho | processo,cunha | cassacao | etica | pinato | camara
1,sp | sao | paulo | brasil | folha,sp | leitorgrupofolhacombr | paulo | sao | lei...
2,temer | presidente | joesley | jbs | ele,joesley | jbs | loures | batista | saud
3,indice | china | economia | pontos | crescimento,indice | yuan | xangai | china | fed
4,indigenas | governo | area | terras | segundo,indigenas | funai | indios | terras | desmatam...
5,us | bilhoes | empresa | grupo | milhoes,inbev | sabmiller | ab | yahoo | kroton
6,anos | mortes | veja | anuncios | filhos,colunaobituariogrupofolhacombr | anuncios | mi...


In [26]:
df_words5_clusters.iloc[clusters_sample, 1:]

,cluster_words_weight,cluster_words_relevance
60,presidente | camara | temer | governo | deputados,maia | centrao | camara | deputados | rosso
84,pessoas | samarco | segundo | ainda | disse,samarco | barragem | lama | terremoto | barragens
92,trabalho | trabalhadores | governo | empresas ...,esocial | terceirizacao | trabalhador | empreg...
45,saude | medicos | hospital | diz | anos,saude | medicos | sus | hospital | pacientes
168,dilma | presidente | contra | impeachment | ma...,manifestantes | mbl | ato | protesto | manifes...
50,policia | segundo | ele | disse | sao,policia | bombeiros | corpo | incendio | iml
71,turquia | erdogan | governo | turco | pais,erdogan | turquia | turco | gulen | recep
113,show | banda | rock | rio | festival,rock | banda | show | palco | shows
87,partido | governo | pais | presidente | eleicoes,macron | pen | fillon | le | marine
140,cerveja | vinho | tel | sao | vinhos,cerveja | vinhos | vinho | ml | cervejas


In [ ]:
topwords_relevance_df_wals_200.T.sample(10).T

In [20]:
topwords_relevance_df_wals_200.T.sample(10).T

,60,84,92,45,168,50,71,113,87,140
0,maia,samarco,esocial,saude,manifestantes,policia,erdogan,rock,macron,cerveja
1,centrao,barragem,terceirizacao,medicos,mbl,bombeiros,turquia,banda,pen,vinhos
2,camara,lama,trabalhador,sus,ato,corpo,turco,show,fillon,vinho
3,deputados,terremoto,empregador,hospital,protesto,incendio,gulen,palco,le,ml
4,rosso,barragens,trabalhadores,pacientes,manifestacao,iml,recep,shows,marine,cervejas
5,temer,fundao,fgts,paciente,protestos,hospital,tayyip,festival,rajoy,alcoolico
6,dem,tremor,trabalhista,medico,impeachment,acidente,pkk,lollapalooza,psoe,bebida
7,bancada,rejeitos,empregado,ans,dilma,delegado,ancara,turne,afd,rotulos
8,demrj,mineradora,trabalhistas,hospitais,manifestacoes,vitima,istambul,in,catalunha,lupulo
9,psb,bhp,clt,cirurgia,cut,morreu,turca,vocalista,keiko,cervejaria


In [18]:
df_words5_clusters.iloc[:60].reset_index()

,index,n_instances,cluster_words_weight,cluster_words_relevance
0,0,766,cunha | camara | presidente | conselho | processo,cunha | cassacao | etica | pinato | camara
1,1,868,sp | sao | paulo | brasil | folha,sp | leitorgrupofolhacombr | paulo | sao | lei...
2,2,840,temer | presidente | joesley | jbs | ele,joesley | jbs | loures | batista | saud
3,3,757,indice | china | economia | pontos | crescimento,indice | yuan | xangai | china | fed
4,4,737,indigenas | governo | area | terras | segundo,indigenas | funai | indios | terras | desmatam...
5,5,884,us | bilhoes | empresa | grupo | milhoes,inbev | sabmiller | ab | yahoo | kroton
6,6,679,anos | mortes | veja | anuncios | filhos,colunaobituariogrupofolhacombr | anuncios | mi...
7,7,1046,presidente | governo | temer | ministro | dilma,ebc | nomeacao | cargo | temer | ministro
8,8,353,degc | sao | temperaturas | temperatura | paulo,degc | temperaturas | termometros | temperatur...
9,9,2853,governo | dilma | presidente | politica | brasil,dilma | impeachment | politica | pt | lula


In [54]:
os.makedirs('outputs')
df_words5_clusters.to_csv("outputs/articles_clusters.csv", index=False)

In [59]:
topwords_weight_df_wals_200.iloc[:, 40:48]

,40,41,42,43,44,45,46,47
0,federal,inflacao,governo,tel,pode,saude,estadio,eu
1,carf,ano,concessoes,sao,anos,medicos,sao,ele
2,lula,juros,concessao,casa,pessoas,hospital,torcedores,anos
3,zelotes,bc,aeroportos,cozinha,saude,diz,futebol,mas
4,operacao,taxa,obras,restaurante,diz,anos,jogo,me
5,empresa,banco,empresas,chef,cancer,sao,paulo,ela
6,ministerio,central,bilhoes,no,mais,medico,corinthians,voce
7,marcondes,meta,investimentos,paulo,estudo,segundo,clube,minha
8,pf,politica,ano,pratos,ainda,mais,palmeiras,dia
9,disse,economia,segundo,onde,pacientes,pacientes,partida,vida


# Revisão

In [20]:
art = pd.read_csv("outputs/articles_clusters_2.csv")

In [21]:
art.loc[:, ['Assunto', 'Subcategoria']] = art[['Assunto', 'Subcategoria']].applymap(lambda x: 'esportes' if x == 'esporte' else x)

In [22]:
art.loc[:, ['Assunto', 'Subcategoria']] = art[['Assunto', 'Subcategoria']].applymap(lambda x: 'transportes' if x == 'transporte' else x)

In [23]:
art.loc[:, ['Assunto', 'Subcategoria']] = art[['Assunto', 'Subcategoria']].applymap(lambda x: 'midias' if x == 'midia' else x)

In [24]:
art = art.rename(columns={'Assunto':'assunto_principal', 'Subcategoria':'assunto_secundario'})

In [25]:
art = art.join(df_words5_clusters.n_instances)

In [27]:
resum_art = art.groupby("assunto_principal").agg({'n_instances':'sum',
                                      'cluster': 'nunique',
                                      'assunto_secundario':'nunique'}
                                    ).sort_values("n_instances", ascending=False)

resum_art.columns = ["n_instances", 'n_cluster', 'n_assunto_secundario']
resum_art.head(30)

,n_instances,n_cluster,n_assunto_secundario
assunto_principal,,,
politica,24266,26,18
esportes,19347,24,21
mundo,10582,13,12
economia,9616,10,9
midias,8212,11,5
indefinido,7081,11,4
crimes,6012,7,5
tv e cinema,5113,4,4
saude,4273,4,4


In [33]:
articles.columns

NameError: name 'articles' is not defined

In [32]:
art.query("assunto_principal == 'governo'")

,cluster,cluster_words_weight,cluster_words_relevance,assunto_principal,assunto_secundario,n_instances
100,100,governo | bilhoes | receita | imposto | ano,imposto | arrecadacao | aliquota | icms | receita,governo,impostos,977


In [49]:
art.groupby("assunto_principal").n_instances.sum().sort_values(ascending=False).head()

assunto_principal
politica    24266
esportes    19347
mundo       10582
economia     9616
midias       8212
Name: n_instances, dtype: int64

In [43]:
assunto_unique = art.Assunto.unique()
assunto_unique

AttributeError: 'DataFrame' object has no attribute 'Assunto'

In [94]:
pd.DataFrame({'ass_princ': art.assunto_principal.value_counts().iloc[:20].index,
              'ass_sec': art.assunto_secundario.value_counts().iloc[:20].index}).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
ass_princ,politica,esportes,mundo,midias,indefinido,economia,crimes,educacao,saude,sociedade,tv e cinema,transportes,musica,negocios,mundo,europa,industria,literatura,impostos,mobilidade
ass_sec,indefinido,jornal,sao paulo,temer,empresas,financas,eleicoes,montadoras,eua,dilma,prisao,presidente,cantareira,lava jato,petrobras,impeachment,escolas,policia,automobilismo,petroleo


In [97]:
(art == 'esportes').any()[-2:]

assunto_principal     True
assunto_secundario    True
dtype: bool

In [130]:
esportes = art.query("assunto_principal == 'esportes' | assunto_secundario == 'esportes'")

In [133]:
data.category.unique()

array(['poder', 'ilustrada', 'mercado', 'mundo', 'esporte', 'tec',
       'cotidiano', 'ambiente', 'equilibrioesaude', 'sobretudo',
       'colunas', 'educacao', 'tv', 'banco-de-dados', 'opiniao',
       'ciencia', 'paineldoleitor', 'saopaulo', 'ilustrissima',
       'seminariosfolha', 'turismo', 'empreendedorsocial', 'serafina',
       'asmais', 'o-melhor-de-sao-paulo', 'bbc', 'comida', 'musica',
       'folhinha', 'especial', 'treinamento', 'multimidia',
       'cenarios-2017', 'topofmind', 'dw', 'ombudsman', 'contas-de-casa',
       'mulher', '2016', 'guia-de-livros-discos-filmes',
       'treinamentocienciaesaude', 'rfi', 'vice', 'bichos', 'euronews',
       'guia-de-livros-filmes-discos', 'infograficos', '2015'],
      dtype=object)

In [143]:
esportes.groupby('assunto_secundario').n_instances.sum().sort_values(ascending=False)

assunto_secundario
futebol               1635
torcida               1626
futebol paulista      1489
olimpiadas            1452
panamericano          1166
futebol espanhol      1122
selecao brasileira    1108
cbf                   1099
volei                  925
tenis                  863
spfc                   851
automobilismo          770
corinthians            755
brasileirao            742
copa libertadores      709
natacao                670
futebol ingles         626
esportes               537
doping                 530
futebol tatica         461
nba                    419
surf                   329
Name: n_instances, dtype: int64

In [131]:
esportes

,cluster,cluster_words_weight,cluster_words_relevance,assunto_principal,assunto_secundario,n_instances
13,13,doping | atletas | antidoping | rio | jogos,doping | antidoping | wada | iaaf | ufc,esportes,doping,530
20,20,surfe | mundial | medina | etapa | mineirinho,surfe | mineirinho | medina | fanning | filipinho,esportes,surf,329
38,38,rio | jogos | comite | olimpiada | olimpico,jogos | comite | olimpico | olimpiada | coi,esportes,olimpiadas,1452
46,46,estadio | sao | torcedores | futebol | jogo,torcedores | estadio | torcida | palmeiras | c...,esportes,torcida,1626
52,52,corinthians | time | tite | contra | equipe,corinthians | tite | jadson | uendel | guerrero,esportes,corinthians,755
58,58,selecao | brasil | rio | jogo | jogos,volei | selecao | jogadoras | bernardinho | ba...,esportes,volei,925
78,78,pontos | campeonato | gol | rodada | city,manchester | city | chelsea | leicester | arsenal,esportes,futebol ingles,626
83,83,selecao | copa | contra | dunga | brasil,dunga | selecao | eliminatorias | copa | neymar,esportes,selecao brasileira,1108
101,101,paulo | sao | time | contra | clube,ceni | muricy | bauza | osorio | ganso,esportes,spfc,851
107,107,sets | ranking | bellucci | parciais | atp,bellucci | sets | atp | parciais | djokovic,esportes,tenis,495


In [139]:
data.query("cluster_wals_200 == 146").sample(10)

,title,text,date,category,subcategory,link,cluster_wals_200
63657,messi errou metade penaltis cobrou temporada,o atacante lionel messi errou dos penaltis cu...,2016-06-27,esporte,None,http://www1.folha.uol.com.br/esporte/2016/06/1...,146
137728,neymar barca iniciam serie decisoes triplice c...,em segundo ano europa neymar tenta ganhar nest...,2015-05-17,esporte,None,http://www1.folha.uol.com.br/esporte/2015/05/1...,146
61396,cesse que musa antiga canta,contra franca seus batalhoes mil vozes entoan...,2016-11-07,colunas,jucakfouri,http://www1.folha.uol.com.br/colunas/jucakfour...,146
90039,juiz determina neymar pai paguem r mil receit...,um juiz federal sao paulo decidiu o atacante n...,2016-01-27,esporte,None,http://www1.folha.uol.com.br/esporte/2016/01/1...,146
156171,lucas silva pode titular real madrid contra sc...,recemcontratado cruzeiro volante pode estrear ...,2015-02-18,esporte,None,http://www1.folha.uol.com.br/esporte/2015/02/1...,146
96655,neymar corre campo continua duvida o mundial,pela primeira vez desde o barcelona desembarco...,2015-12-16,esporte,None,http://www1.folha.uol.com.br/esporte/2015/12/1...,146
139487,entenda greve patronal abala futebol espanhol,a federacao espanhola futebol anunciou suspens...,2015-07-05,bbc,None,http://www1.folha.uol.com.br/bbc/2015/05/16259...,146
162402,corinthians deixa lista clubes maior faturamen...,sem titulos corinthians deixou relacao clube...,2015-01-22,esporte,None,http://www1.folha.uol.com.br/esporte/2015/01/1...,146
98570,dunga pede menos paternalismo jogadores declar...,tecnico selecao brasileira dunga criticou tipo...,2015-06-12,esporte,None,http://www1.folha.uol.com.br/esporte/2015/12/1...,146
81182,uniao europeia confirma proibicao fifa investi...,o tribunal apelacoes uniao europeia recusou ap...,2016-03-17,esporte,None,http://www1.folha.uol.com.br/esporte/2016/03/1...,146


Problemas de Esportes:
+ Coocorrencia muda em funcao de atletas que geralmente são entidades chave, sobretudo no futebol internacional que é consumido por brasileiros para acompanhar os craques do país, como neymar. 
+ 

In [152]:
print(data.query("cluster_wals_200 == 114").sample(10).title)

139729    clubes brasileiros vetaram arbitros pais oitav...
142948    clima tenso marca semi vasco fla rivais decide...
21246     principal aposta palmeiras borja a prova fogo ...
143398    com empate gols corinthians classifica liberta...
25057     cbf define datas horarios jogos quarta fase co...
98414       com vaga libertadores sao paulo maratona jogos 
46518             populacao poder compra brasil decisivos g
118317    palmeiras pega cruzeiro manter retrospecto ani...
86865     abismo financeiro separa brasileiros clubes ri...
24122     meia paraguai enfrentou propria torcida contra...
Name: title, dtype: object


In [149]:
data.query("cluster_wals_200 == 177").sample(10)

,title,text,date,category,subcategory,link,cluster_wals_200
113139,cbf pede explicacoes sao paulo triangulacao co...,o sao paulo que explicar a cbf foi processo co...,2015-09-22,esporte,None,http://www1.folha.uol.com.br/esporte/2015/09/1...,177
16351,sport anuncia contratacao tecnico vanderlei lu...,o sport anunciou nesta segundafeira contratac...,2017-05-29,esporte,None,http://www1.folha.uol.com.br/esporte/2017/05/1...,177
82221,empresario cuca palmeiras reunemse discutir co...,neste domingo diretoria palmeiras o represent...,2016-12-03,esporte,None,http://www1.folha.uol.com.br/esporte/2016/03/1...,177
19391,jogadores brasil processam produtoras games di...,mais jogadores futebol brasileiros ganharam a...,2017-05-05,tec,None,http://www1.folha.uol.com.br/tec/2017/05/18813...,177
110161,presidente conselho diz nao sabe aidar termina...,em meio crise politica sao paulo presidente co...,2015-08-10,esporte,None,http://www1.folha.uol.com.br/esporte/2015/10/1...,177
33875,felipe melo fara exames palmeiras deve anuncia...,o volante felipe melo chegou sao paulo nesta ...,2017-03-01,esporte,None,http://www1.folha.uol.com.br/esporte/2017/01/1...,177
143493,muricy faz cirurgia a retirada vesicula passa bem,dez dias apos deixar sao paulo tecnico muricy ...,2015-04-16,esporte,None,http://www1.folha.uol.com.br/esporte/2015/04/1...,177
150809,santos efetiva marcelo fernandes tecnico time,interino desde demissao enderson moreira marce...,2015-12-03,esporte,None,http://www1.folha.uol.com.br/esporte/2015/03/1...,177
42521,atacante jo assina o corinthians tres temporadas,o atacante jo assinou contrato valido tres te...,2016-10-31,esporte,None,http://www1.folha.uol.com.br/esporte/2016/10/1...,177
148919,artilheiro tem emprego garantido apos final ca...,nem luis fabiano alexandre pato nem guerrero o...,2015-03-20,esporte,None,http://www1.folha.uol.com.br/esporte/2015/03/1...,177


In [153]:
data.query("cluster_wals_200 == 52").sample(10)

,title,text,date,category,subcategory,link,cluster_wals_200
45640,vilson acerta soco rosto marciel treino corint...,o zagueiro vilson acertou soco rosto volante m...,2016-11-10,esporte,None,http://www1.folha.uol.com.br/esporte/2016/10/1...,52
134904,perdemos espinha dorsal diz renato augusto a s...,apos derrota o palmeiras neste domingo itaq...,2015-05-31,esporte,None,http://www1.folha.uol.com.br/esporte/2015/05/1...,52
155446,com lesao joelho fabio santos desfalca corinth...,na manha deste sabado corinthians anunciou se...,2015-02-21,esporte,None,http://www1.folha.uol.com.br/esporte/2015/02/1...,52
19087,corinthians fatura titulo apostas base desconh...,desconhecidos contestados apostas jogadores ba...,2017-07-05,esporte,None,http://www1.folha.uol.com.br/esporte/2017/05/1...,52
145475,comparado idolo santista vladimir contrasta vi...,o ceu o inferno dois goleiros desempenhos opos...,2015-06-04,esporte,None,http://www1.folha.uol.com.br/esporte/2015/04/1...,52
26481,sao paulo pede federacao nega adiamento jogo c...,o adiamento classico sao paulo corinthians mar...,2017-07-03,esporte,None,http://www1.folha.uol.com.br/esporte/2017/03/1...,52
137935,temos mesmo sentimento torcedor diz tite prote...,nove torcedores organizada camisa fizeram pro...,2015-05-15,esporte,None,http://www1.folha.uol.com.br/esporte/2015/05/1...,52
131025,a gente joga amor tem vagabundo afirma tite ap...,o tecnico tite mostrou certo incomodo apos der...,2015-06-21,esporte,None,http://www1.folha.uol.com.br/esporte/2015/06/1...,52
157088,vagner love se coloca disposicao diz historia ...,apresentado oficialmente nesta sextafeira ata...,2015-02-13,esporte,None,http://www1.folha.uol.com.br/esporte/2015/02/1...,52
60241,pato pode titular no proximo sabado contra fig...,depois danilo testado centroavante classico na...,2016-07-18,esporte,None,http://www1.folha.uol.com.br/esporte/2016/07/1...,52


In [159]:
data.query("cluster_wals_200 == 46").sample(10)

,title,text,date,category,subcategory,link,cluster_wals_200
42408,presidente corinthians diz temer seguranca tor...,roberto andrade presidente corinthians mostrou...,2016-01-11,esporte,None,http://www1.folha.uol.com.br/esporte/2016/11/1...,46
46553,desempenho derruba publico itaquera preocupa c...,de postulante titulo um mero candidato disputa...,2016-05-10,esporte,None,http://www1.folha.uol.com.br/esporte/2016/10/1...,46
91677,em vitoria sao paulo torcedores entram confron...,um grupo torcedores uma torcida organizada sao...,2016-01-17,esporte,None,http://www1.folha.uol.com.br/esporte/2016/01/1...,46
70191,ministerio publico veta torcida anel inferior ...,apos vistoria policia militar ministerio publi...,2016-05-19,esporte,None,http://www1.folha.uol.com.br/esporte/2016/05/1...,46
75414,acho feio e legal diz cuca torcida unica semif...,do ponto vista esportivo medida secretaria seg...,2016-04-19,esporte,None,http://www1.folha.uol.com.br/esporte/2016/04/1...,46
90021,mogi cruzes cobra sao paulo federacao danos es...,a prefeitura mogi cruzes encaminhou notificaca...,2016-01-27,esporte,None,http://www1.folha.uol.com.br/esporte/2016/01/1...,46
105813,brasileiro realiza sonho jogar wembley como jo...,o estadio wembley londres inglaterra um mais c...,2015-10-30,esporte,None,http://www1.folha.uol.com.br/esporte/2015/10/1...,46
158928,tumulto arquibancada interrompe semifinal copa...,pais sede copa africana nacoes guine equatoria...,2015-05-02,esporte,None,http://www1.folha.uol.com.br/esporte/2015/02/1...,46
142735,corinthians diz cadeiras quebradas torcida rival,o corinthians anunciou nesta segundafeira ca...,2015-04-20,esporte,None,http://www1.folha.uol.com.br/esporte/2015/04/1...,46
9652,hernanes reencontra sao paulo crise tenta ofer...,quando meiocampista hernanes deixou sao paulo ...,2017-07-25,esporte,None,http://www1.folha.uol.com.br/esporte/2017/07/1...,46


In [172]:
data.query("subcategory == 'olimpiada-no-rio'").cluster_wals_200.value_counts().head(20)

38     613
129    519
58     349
123    269
152    236
56     152
13     132
83     120
185     54
106     28
147     27
157     26
46      26
142     25
36      24
67      14
190     13
47      11
29      10
7       10
Name: cluster_wals_200, dtype: int64

In [174]:
art.join(data.query("subcategory == 'olimpiada-no-rio'").cluster_wals_200.value_counts().head(10), how='right')

,cluster,cluster_words_weight,cluster_words_relevance,assunto_principal,assunto_secundario,n_instances,cluster_wals_200
38,38,rio | jogos | comite | olimpiada | olimpico,jogos | comite | olimpico | olimpiada | coi,esportes,olimpiadas,1452,613
129,129,rio | brasil | jogos | atletas | ouro,medalhas | medalha | bronze | judo | pan,esportes,panamericano,1166,519
58,58,selecao | brasil | rio | jogo | jogos,volei | selecao | jogadoras | bernardinho | ba...,esportes,volei,925,349
123,123,mundial | rio | prova | livre | jogos,cielo | bolt | natacao | kazan | medley,esportes,natacao,670,269
152,152,the | of | in | to | and,the | to | in | that | of,inglês,inglês,240,236
56,56,la | el | en | los | del,en | el | juegos | los | un,espanhol,indefinido,173,152
13,13,doping | atletas | antidoping | rio | jogos,doping | antidoping | wada | iaaf | ufc,esportes,doping,530,132
83,83,selecao | copa | contra | dunga | brasil,dunga | selecao | eliminatorias | copa | neymar,esportes,selecao brasileira,1108,120
185,185,tenis | ranking | torneio | disse | aberto,tenista | slam | tenis | nadal | garros,esportes,tenis,368,54
106,106,federal | operacao | policia | prisao | pf,cabral | pf | eike | mandados | operacao,crimes,prisao,1077,28


Explicar aqui que a diferença entre clusterização e classificação em termos de precisão...

## Politicas

In [178]:
(art == 'politica').any()

cluster                    False
cluster_words_weight       False
cluster_words_relevance    False
assunto_principal           True
assunto_secundario         False
n_instances                False
dtype: bool

In [179]:
politica = art.query("assunto_principal == 'politica'")

In [181]:
politica

,cluster,cluster_words_weight,cluster_words_relevance,assunto_principal,assunto_secundario,n_instances
0,0,cunha | camara | presidente | conselho | processo,cunha | cassacao | etica | pinato | camara,politica,cunha,766
2,2,temer | presidente | joesley | jbs | ele,joesley | jbs | loures | batista | saud,politica,jbs,840
7,7,presidente | governo | temer | ministro | dilma,ebc | nomeacao | cargo | temer | ministro,politica,presidente,1046
9,9,governo | dilma | presidente | politica | brasil,dilma | impeachment | politica | pt | lula,politica,pt,2853
16,16,cunha | contas | milhoes | suica | dinheiro,suica | cunha | offshore | zelada | contas,politica,cunha,492
27,27,cpi | presidente | disse | cunha | petrobras,cpi | coaf | capez | requerimentos | merenda,politica,cpi,540
40,40,federal | carf | lula | zelotes | operacao,carf | zelotes | marcondes | mautoni | aps,politica,lula,305
48,48,stf | federal | petrobras | jato | lava,youssef | inquerito | stf | janot | doleiro,politica,lava jato,805
60,60,presidente | camara | temer | governo | deputados,maia | centrao | camara | deputados | rosso,politica,temer,824
61,61,partidos | politica | camara | eleitoral | ref...,distritao | partidos | coligacoes | candidatos...,politica,eleicoes,725


In [184]:
politica.groupby("assunto_secundario").n_instances.sum().sort_values(ascending=False)

assunto_secundario
pt                4080
temer             2397
impeachment       2218
presidente        2101
eleicoes          1551
sao paulo         1422
stf               1313
cunha             1258
camara            1133
senado             973
lula               953
rio de janeiro     872
jbs                840
lava jato          805
mercosul           704
ministerio         665
cpi                540
dilma              441
Name: n_instances, dtype: int64

In [ ]:
politica.groupby("assunto_secundario").n_instances.sum().sort_values(ascending=False)

## Mundo

In [185]:
(art == 'mundo').any()

cluster                    False
cluster_words_weight       False
cluster_words_relevance    False
assunto_principal           True
assunto_secundario         False
n_instances                False
dtype: bool

In [186]:
mundo = art.query("assunto_principal == 'mundo'")
mundo

,cluster,cluster_words_weight,cluster_words_relevance,assunto_principal,assunto_secundario,n_instances
21,21,mas | mundo | mais | vida | sua,mundo | vida | freud | filosofia | mas,mundo,filosofia,1774
82,82,siria | russia | disse | sirio | rebeldes,siria | aleppo | sirio | assad | rebeldes,mundo,siria,663
91,91,ei | estado | islamico | siria | iraque,ei | mossul | iraque | islamico | iraquianas,mundo,estado islamico,643
115,115,pais | ataque | pessoas | contra | disse,iemen | taleban | houthis | saudita | hadi,mundo,ataques,725
122,122,indonesia | brasileiro | brasil | governo | morte,pizzolato | gularte | indonesia | chapo | archer,mundo,indonesia,327
128,128,trump | presidente | disse | governo | russia,trump | comey | flynn | putin | fbi,mundo,eua,847
137,137,argentina | macri | cristina | governo | presi...,macri | cristina | kirchner | nisman | scioli,mundo,argentina,966
139,139,maduro | venezuela | governo | oposicao | pres...,maduro | venezuela | caracas | chavista | nicolas,mundo,venezuela,996
153,153,israel | palestinos | israelense | israelenses...,israel | palestinos | israelenses | israelense...,mundo,israel,745
163,163,trump | hillary | disse | campanha | republicano,hillary | trump | republicano | sanders | clinton,mundo,eua,1440


In [101]:
esportes.assunto_secundario

13                 doping
20                   surf
38             olimpiadas
46                torcida
52            corinthians
58                  volei
78         futebol ingles
83     selecao brasileira
101                  spfc
107                 tenis
111                   cbf
114     copa libertadores
118         automobilismo
123               natacao
124           brasileirao
129          panamericano
138        futebol tatica
146      futebol espanhol
147                   nba
177      futebol paulista
182               futebol
185                 tenis
190               futebol
191              esportes
195         automobilismo
Name: assunto_secundario, dtype: object

In [75]:
sub_assunto_unique = art.Subcategoria.unique()
sub_assunto_unique

array(['cunha', 'jornal', 'jbs', 'china', 'desmatamento', 'empresas',
       'missas', 'presidente', 'sao paulo', 'pt', 'financas', 'blog',
       'romance', 'doping', 'contabilidade', 'decoracao', 'escolas',
       'sorteio', 'editora', 'surf', 'filosofia', 'previdencia',
       'reino unido', 'reciclagem', 'naufragio', 'cpi', 'policia',
       'inflacao', 'sociedade', 'vestibular', 'facebook', 'mulheres',
       'europa', 'uber', 'olimpiadas', 'rodovias', 'lula',
       'infraestrutura', 'restaurante', 'doencas', 'sus', 'torcida',
       'indefinido', 'lava jato', 'greve', 'incendio', 'tirinhas',
       'corinthians', 'petroleo', 'grecia', 'refugiados', 'volei',
       'tecnologia', 'temer', 'eleicoes', 'exposicao', 'ministerio',
       'catolica', 'metro', 'dilma', 'prisao', 'turquia', 'moda',
       'aquecimento global', 'rio de janeiro', 'futebol ingles', 'series',
       'camara', 'coluna', 'siria', 'selecao brasileira', 'barragem',
       'aeroportos', 'estado islamico', 'trabal

In [68]:
assunto unique 2 
for assunto in assunto_unique:
    

array(['politica', 'midia', 'economia', 'indigenas', 'negocios', 'mortes',
       'tempo', 'frederico vasconcelos', 'literatura', 'esporte',
       'impostos', 'casa', 'educacao', 'megasena', 'mundo', 'europa',
       'imoveis', 'sustentabilidade', 'nautica', 'crimes',
       'redes sociais', 'assedio', 'terrorismo', 'transporte', 'esportes',
       'acidentes', 'gastronomia', 'saude', 'pronomes', 'sindicatos',
       'policia', 'cartum', 'espanhol', 'mundo ', 'diversos', 'arte',
       'religiao', 'moda', 'indefinido', 'tv e cinema', 'samarco',
       'transportes', 'sociedade', 'consumidor', 'financas', 'mobilidade',
       'artes', 'midias', 'governo', 'musica', 'industria',
       'empreendedorismo', 'telefonia', 'turismo', 'ciencia', 'lava jato',
       'abastecimento', 'bebidas', 'presidios', 'cidade', 'inglês',
       'cinema', 'infantil', 'sao paulo', 'drogas', 'animais',
       'petrobras', 'tecnologia', 'eleicoes', 'energia', 'carnaval'],
      dtype=object)

In [163]:
print(art.query("assunto_principal =='esportes'").assunto_secundario.values)

['doping' 'surf' 'olimpiadas' 'torcida' 'corinthians' 'volei'
 'futebol ingles' 'selecao brasileira' 'spfc' 'tenis' 'cbf'
 'copa libertadores' 'automobilismo' 'natacao' 'brasileirao'
 'panamericano' 'futebol tatica' 'futebol espanhol' 'nba'
 'futebol paulista' 'futebol' 'tenis' 'futebol' 'automobilismo']


In [157]:
resum_art.head(8)

,n_instances,n_cluster,n_assunto_secundario
assunto_principal,,,
politica,24266,26,18
esportes,19347,24,21
mundo,10582,13,12
economia,9616,10,9
midias,8212,11,5
indefinido,7081,11,4
crimes,6012,7,5
tv e cinema,5113,4,4


In [53]:
cluster=20
pd.DataFrame(data.query("cluster_wals_50 == @cluster").cluster_wals_200.value_counts()).join(art).head(10)

,cluster_wals_200,cluster,cluster_words_weight,cluster_words_relevance,assunto_principal,assunto_secundario,n_instances
79,1589,79,serie | tv | anos | temporada | programa,netflix | serie | hbo | thrones | episodios,tv e cinema,series,1657
183,128,183,melhor | filme | oscar | ator | atriz,oscar | coadjuvante | melhor | birdman | atriz,tv e cinema,oscar,696
154,109,154,filme | anos | us | filmes | milhoes,wars | star | filme | marvel | disney,cinema,hollywood,1081
81,51,81,leia | aqui | coluna | completa | diz,coluna | leia | completa | aqui | diz,midias,coluna,1218
131,49,131,jornal | folha | anos | brasil | jornalismo,jornalismo | jornal | jornais | nyt | jornalistas,midias,jornal,1349
47,29,47,eu | ele | anos | mas | me,eu | me | pra | minha | meu,pronomes,indefinido,2237
34,25,34,facebook | usuarios | empresa | pessoas | rede,facebook | usuarios | whatsapp | google | apli...,redes sociais,facebook,1400
120,21,120,oi | internet | empresa | brasil | anatel,oi | anatel | operadoras | telefonia | telecom...,telefonia,oi,528
135,20,135,filme | anos | cinema | mas | sua,filme | cinema | elenco | filmes | tc,tv e cinema,cinema,1604
108,17,108,texto | erramos | corrigido | diferentemente |...,erramos | corrigido | diferentemente | impress...,midias,erros,444


In [41]:
df_words5_clusters_50

,cluster,n_instances,cluster_words_weight,cluster_words_relevance
0,0,2817,empresa | apple | google | disse | us,apple | google | iphone | android | usuarios
1,1,3744,governo | presidente | pais | disse | venezuela,maduro | venezuela | caracas | coreia | venezu...
2,2,1976,brasil | para | sua | pais | participacao,traduzem | obedece | contemporaneo | refletir ...
3,3,4905,time | selecao | contra | clube | jogador,jogador | neymar | selecao | jogadores | palme...
4,4,434,voce | hoje | lua | seu | pode,jul | fev | abr | nov | lua
5,5,2868,brasil | folha | leitores | no | para,enviando | envie | leitoruolcombr | leitores |...
6,6,964,sportv | espn | brasil | nba | copa,sportv | espn | nba | warriors | sports
7,7,3737,anos | pode | saude | mais | diz,virus | zika | cientistas | pesquisadores | mi...
8,8,4780,governo | ano | bilhoes | presidente | anos,governo | bilhoes | imposto | receita | previd...
9,9,1967,folha | brasil | sao | paulo | social,empreendedor | inscricoes | pags | empreendedo...


# Apresentação Final

In [1]:
topwords_relevance_df_wals_200

NameError: name 'topwords_relevance_df_wals_200' is not defined

In [69]:
cluster=44
pd.DataFrame(data.query("cluster_wals_50 == @cluster").cluster_wals_200.value_counts()).join(art).head(10)

,cluster_wals_200,cluster,cluster_words_weight,cluster_words_relevance,assunto_principal,assunto_secundario,n_instances
12,1261,12,livro | autor | anos | sua | seu,livro | romance | autor | literatura | pags,literatura,romance,1363
55,1206,55,livro | anos | livros | editora | autor,flip | editora | livros | livro | literatura,literatura,editora,1274
155,328,155,criancas | anos | pais | crianca | diz,criancas | crianca | folhinha | folhinhauolcom...,infantil,midias,1050
47,181,47,eu | ele | anos | mas | me,eu | me | pra | minha | meu,pronomes,indefinido,2237
64,101,64,arte | artista | obras | museu | anos,arte | artista | museu | exposicao | galeria,arte,exposicao,1844
131,89,131,jornal | folha | anos | brasil | jornalismo,jornalismo | jornal | jornais | nyt | jornalistas,midias,jornal,1349
102,51,102,anos | musica | sao | paulo | diz,musica | disco | compositor | samba | orquestra,musica,indefinido,1511
21,49,21,mas | mundo | mais | vida | sua,mundo | vida | freud | filosofia | mas,mundo,filosofia,1774
154,45,154,filme | anos | us | filmes | milhoes,wars | star | filme | marvel | disney,cinema,hollywood,1081
105,28,105,disse | anos | pessoas | ele | pais,negros | negro | brancos | racismo | racial,sociedade,racismo,1745


# Clusterização de Busca Extensiva - 50 Clusters

In [7]:
clustering_outputs_wals_50  = np.load('models/clusters_wals_unigram_articles_50_outputs.npz')
data['cluster_wals_50'] = clustering_outputs_wals_50['index']

In [8]:
%%time
word_cluster_wals_50 = TopicClustering.get_word_cluster_matrix(data.cluster_wals_50, tf).astype('float32')
topwords_relevance_df_wals_50, top5_words_relevance_wals_50 = TopicClustering.view_topics(word_cluster_wals_50, vocabulary, relevance=True, n_top_words=50)
topwords_weight_df_wals_50, top5_words_weight_wals_50       = TopicClustering.view_topics(word_cluster_wals_50, vocabulary, relevance=False, n_top_words=50)

CPU times: user 807 ms, sys: 15.9 ms, total: 823 ms
Wall time: 821 ms


In [9]:
df_words5_clusters_50 = pd.DataFrame({'n_instances': data.cluster_wals_50.value_counts(),
                                      "cluster_words_weight": top5_words_weight_wals_50, 
                                      "cluster_words_relevance": top5_words_relevance_wals_50})

In [17]:
#df_words5_clusters_50['cluster'] = df_words5_clusters_50.index
df_words5_clusters_50 = df_words5_clusters_50.iloc[:,[ 3, 0, 1,2]]

In [19]:
df_words5_clusters_50.to_csv("outputs/articles_clustering_extensive.csv", index=False)

# Resgatando Rotulação da Clusterização Extensiva

In [71]:
df_extensive = pd.read_csv("outputs/articles_clustering_extensive.csv")

In [109]:
df_extensive = df_extensive.applymap(lambda x: 'indefinido' if x == 'indefinido ' else x)

In [110]:
df_extensive.assunto_princ.nunique()

25

In [95]:
print(df_extensive.assunto_princ.unique())

['tecnologia' 'mundo ' 'brasil' 'esportes' 'indefinido' 'midias' 'saude'
 'governo ' 'ingles' 'sindicatos' 'economia' 'politica' 'gastronomia'
 'moda' 'tv e cinema' 'educacao' 'arte' 'acidentes' 'transportes'
 'lava jato' 'sao paulo ' 'policia' 'musica' 'literatura' 'indefinido '
 'espanhol']


In [107]:
pd.Series(df_extensive.assunto_princ.unique()).sort_values().values

array(['acidentes', 'arte', 'brasil', 'economia', 'educacao', 'espanhol',
       'esportes', 'gastronomia', 'governo ', 'indefinido', 'indefinido ',
       'ingles', 'lava jato', 'literatura', 'midias', 'moda', 'mundo ',
       'musica', 'policia', 'politica', 'sao paulo ', 'saude',
       'sindicatos', 'tecnologia', 'transportes', 'tv e cinema'],
      dtype=object)

In [111]:
print(pd.Series(df_extensive.assunto_princ.unique()).sort_values().values)

['acidentes' 'arte' 'brasil' 'economia' 'educacao' 'espanhol' 'esportes'
 'gastronomia' 'governo ' 'indefinido' 'ingles' 'lava jato' 'literatura'
 'midias' 'moda' 'mundo ' 'musica' 'policia' 'politica' 'sao paulo '
 'saude' 'sindicatos' 'tecnologia' 'transportes' 'tv e cinema']


In [ ]:
'acidentes' 'arte' 'brasil' 'economia' 'educacao' 'espanhol' 'esportes'
 'gastronomia' 'governo ' 'indefinido' 'indefinido ' 'ingles' 'lava jato'
 'literatura' 'midias' 'moda' 'mundo ' 'musica' 'policia' 'politica'
 'sao paulo ' 'saude' 'sindicatos' 'tecnologia' 'transportes'

In [116]:
df_extensive.head(2)

,cluster,n_instances,cluster_words_weight,cluster_words_relevance,assunto_princ,assunto_sec
0,0,2817,empresa | apple | google | disse | us,apple | google | iphone | android | usuarios,tecnologia,empresas
1,1,3744,governo | presidente | pais | disse | venezuela,maduro | venezuela | caracas | coreia | venezu...,mundo,venezuela


In [117]:
resum_ext = df_extensive.groupby("assunto_princ").agg({'n_instances':'sum',
                                                       'cluster': 'nunique',
                                                       'assunto_sec':'nunique'}
                                                      ).sort_values("n_instances", ascending=False)

resum_ext.columns = ["n_instances", 'n_cluster', 'n_assunto_secundario']
resum_ext.head(30)

,n_instances,n_cluster,n_assunto_secundario
assunto_princ,,,
politica,23266,5,4
indefinido,21148,6,2
esportes,19368,7,6
sao paulo,17848,1,1
mundo,13829,5,5
economia,9688,3,3
midias,7101,4,3
tv e cinema,6761,2,2
lava jato,5898,1,1


In [126]:
df_extensive.query("assunto_princ == 'esportes'")

,cluster,n_instances,cluster_words_weight,cluster_words_relevance,assunto_princ,assunto_sec
3,3,4905,time | selecao | contra | clube | jogador,jogador | neymar | selecao | jogadores | palme...,esportes,futebol
6,6,964,sportv | espn | brasil | nba | copa,sportv | espn | nba | warriors | sports,esportes,midias
14,14,1539,ano | ranking | brasileiro | final | mundial,gp | sets | bellucci | djokovic | hamilton,esportes,geral
16,16,2386,gol | time | jogo | pontos | com,min | gol | rodada | placar | gols,esportes,futebol
24,24,3973,futebol | cbf | fifa | presidente | copa,cbf | fifa | futebol | nero | blatter,esportes,federacoes
40,40,1668,sao | paulo | time | contra | clube,ceni | muricy | morumbi | libertadores | bauza,esportes,spfc
45,45,3933,rio | jogos | brasil | mundial | atletas,atletas | jogos | medalhas | medalha | olimpico,esportes,olimpiadas


In [133]:
esportes_extensive = [3, 6, 14, 16, 24, 40, 45]
a = data.query("cluster_wals_50 in @esportes_extensive").cluster_wals_200.value_counts()
a

46     1481
177    1426
129    1160
83     1100
146    1093
       ... 
21        1
37        1
53        1
44        1
90        1
Name: cluster_wals_200, Length: 120, dtype: int64

In [139]:
esportes_data = data.query("cluster_wals_50 in @esportes_extensive")

In [165]:
esp_intensive = pd.DataFrame({'% de noticias':esportes_data.cluster_wals_200.value_counts(normalize=True)*100,
                  'n de noticias':esportes_data.cluster_wals_200.value_counts()}
            ).join(art.iloc[:,:-1]).head(20)

In [171]:
esp_intensive.iloc[:, [2, 0, 1, 3,4,5,6]].rename(columns={'cluster':"intensive_cluster"})

,intensive_cluster,% de noticias,n de noticias,cluster_words_weight,cluster_words_relevance,assunto_principal,assunto_secundario
46,46,7.646634,1481,estadio | sao | torcedores | futebol | jogo,torcedores | estadio | torcida | palmeiras | c...,esportes,torcida
177,177,7.362660,1426,clube | sao | paulo | futebol | corinthians,clube | jogador | aidar | palmeiras | corinthians,esportes,futebol paulista
129,129,5.989261,1160,rio | brasil | jogos | atletas | ouro,medalhas | medalha | bronze | judo | pan,esportes,panamericano
83,83,5.679471,1100,selecao | copa | contra | dunga | brasil,dunga | selecao | eliminatorias | copa | neymar,esportes,selecao brasileira
146,146,5.643329,1093,barcelona | neymar | jogador | messi | clube,barcelona | messi | neymar | madrid | cristiano,esportes,futebol espanhol
111,111,5.627840,1090,fifa | cbf | futebol | presidente | entidade,fifa | cbf | nero | blatter | marin,esportes,cbf
58,58,4.708798,912,selecao | brasil | rio | jogo | jogos,volei | selecao | jogadoras | bernardinho | ba...,esportes,volei
182,182,4.429988,858,palmeiras | contra | time | santos | jogo,palmeiras | dudu | arouca | cuca | robinho,esportes,futebol
101,101,4.393846,851,paulo | sao | time | contra | clube,ceni | muricy | bauza | osorio | ganso,esportes,spfc
190,190,4.011772,777,gol | jogo | time | min | bola,min | gol | bola | goleiro | placar,esportes,futebol


In [152]:
df_extensive.query("assunto_princ == 'esportes'")

,cluster,n_instances,cluster_words_weight,cluster_words_relevance,assunto_princ,assunto_sec
3,3,4905,time | selecao | contra | clube | jogador,jogador | neymar | selecao | jogadores | palme...,esportes,futebol
6,6,964,sportv | espn | brasil | nba | copa,sportv | espn | nba | warriors | sports,esportes,midias
14,14,1539,ano | ranking | brasileiro | final | mundial,gp | sets | bellucci | djokovic | hamilton,esportes,geral
16,16,2386,gol | time | jogo | pontos | com,min | gol | rodada | placar | gols,esportes,futebol
24,24,3973,futebol | cbf | fifa | presidente | copa,cbf | fifa | futebol | nero | blatter,esportes,federacoes
40,40,1668,sao | paulo | time | contra | clube,ceni | muricy | morumbi | libertadores | bauza,esportes,spfc
45,45,3933,rio | jogos | brasil | mundial | atletas,atletas | jogos | medalhas | medalha | olimpico,esportes,olimpiadas


In [172]:
futebol_data = data.query("cluster_wals_50==3")
futebol_intensive = pd.DataFrame({'% de noticias':futebol_data.cluster_wals_200.value_counts(normalize=True)*100,
              'n de noticias':futebol_data.cluster_wals_200.value_counts()}
            ).join(art.iloc[:,:-1]).head(20)

In [174]:
futebol_intensive.iloc[:, [2, 0, 1, 3,4,5,6]].rename(columns={'cluster':"intensive_cluster"})

,intensive_cluster,% de noticias,n de noticias,cluster_words_weight,cluster_words_relevance,assunto_principal,assunto_secundario
177,177,19.082569,936,clube | sao | paulo | futebol | corinthians,clube | jogador | aidar | palmeiras | corinthians,esportes,futebol paulista
146,146,18.695209,917,barcelona | neymar | jogador | messi | clube,barcelona | messi | neymar | madrid | cristiano,esportes,futebol espanhol
83,83,17.166157,842,selecao | copa | contra | dunga | brasil,dunga | selecao | eliminatorias | copa | neymar,esportes,selecao brasileira
182,182,15.290520,750,palmeiras | contra | time | santos | jogo,palmeiras | dudu | arouca | cuca | robinho,esportes,futebol
52,52,10.988787,539,corinthians | time | tite | contra | equipe,corinthians | tite | jadson | uendel | guerrero,esportes,corinthians
138,138,8.073394,396,futebol | time | brasil | bola | jogadores,passes | bola | meiocampo | treinadores | joga...,esportes,futebol tatica
124,124,1.855250,91,pontos | time | brasileiro | rodada | campeonato,rodada | coritiba | avai | gremio | figueirense,esportes,brasileirao
46,46,1.773700,87,estadio | sao | torcedores | futebol | jogo,torcedores | estadio | torcida | palmeiras | c...,esportes,torcida
114,114,1.508665,74,sao | paulo | libertadores | time | corinthians,libertadores | river | amarilla | juniors | plate,esportes,copa libertadores
58,58,1.100917,54,selecao | brasil | rio | jogo | jogos,volei | selecao | jogadoras | bernardinho | ba...,esportes,volei


In [188]:
data.query("cluster_wals_200 == 83").sample(10)

,title,text,date,category,subcategory,link,cluster_wals_200,cluster_wals_50
131083,selecao joga evitar vexame por melhor caminho ...,o jogo deste domingo selecao brasileira contr...,2015-06-21,esporte,None,http://www1.folha.uol.com.br/esporte/2015/06/1...,83,24
131531,depois liga pedir perdao diz neymar jogador o ...,um atrativos a partida brasil colombia nesta q...,2015-06-18,esporte,None,http://www1.folha.uol.com.br/esporte/2015/06/1...,83,3
127616,aniversario um ano a alemanha vira piada int...,o aniversario um ano derrota selecao brasileir...,2015-08-07,esporte,None,http://www1.folha.uol.com.br/esporte/2015/07/1...,83,24
129607,medico selecao diz nao conseguiu identificar c...,o medico selecao brasileira rodrigo lasmar dis...,2015-06-28,esporte,None,http://www1.folha.uol.com.br/esporte/2015/06/1...,83,3
24284,em retaliacao tite fecha treino tatico selecao...,tite fechou treino tatico deste domingo itaqu...,2017-03-26,esporte,None,http://www1.folha.uol.com.br/esporte/2017/03/1...,83,3
126530,fifa nega recurso cbf liberar neymar punicao e...,a comissao disciplinar fifa negou recurso cbf ...,2015-07-14,esporte,None,http://www1.folha.uol.com.br/esporte/2015/07/1...,83,24
73796,cbf divulga preconvocados dunga copa america n...,a cbf promete divulgar nesta sexta jogadores...,2016-04-28,esporte,None,http://www1.folha.uol.com.br/esporte/2016/04/1...,83,3
109835,selecao chega fortaleza jogadores vao direto f...,a selecao brasileira esta fortaleza a partida ...,2015-09-10,esporte,None,http://www1.folha.uol.com.br/esporte/2015/10/1...,83,3
110769,philippe coutinho cortado dunga convoca kaka e...,o tecnico dunga convocou noite deste domingo ...,2015-04-10,esporte,None,http://www1.folha.uol.com.br/esporte/2015/10/1...,83,3
67469,um estorvo chamado selecao,o contra equador um primeiro tempo razoavel ...,2016-05-06,colunas,jucakfouri,http://www1.folha.uol.com.br/colunas/jucakfour...,83,24


In [193]:
data.iloc[[131083,131531, 127616, 129607, 24284, 126530, 73796, 109835, 67469, 110769], [0,1, -1,-2]].rename(columns={'cluster_wals_200': 'cluster_intensive', 
                                                                                                                      'cluster_wals_50': 'cluster_extensive'})

,title,text,cluster_extensive,cluster_intensive
131083,selecao joga evitar vexame por melhor caminho ...,o jogo deste domingo selecao brasileira contr...,24,83
131531,depois liga pedir perdao diz neymar jogador o ...,um atrativos a partida brasil colombia nesta q...,3,83
127616,aniversario um ano a alemanha vira piada int...,o aniversario um ano derrota selecao brasileir...,24,83
129607,medico selecao diz nao conseguiu identificar c...,o medico selecao brasileira rodrigo lasmar dis...,3,83
24284,em retaliacao tite fecha treino tatico selecao...,tite fechou treino tatico deste domingo itaqu...,3,83
126530,fifa nega recurso cbf liberar neymar punicao e...,a comissao disciplinar fifa negou recurso cbf ...,24,83
73796,cbf divulga preconvocados dunga copa america n...,a cbf promete divulgar nesta sexta jogadores...,3,83
109835,selecao chega fortaleza jogadores vao direto f...,a selecao brasileira esta fortaleza a partida ...,3,83
67469,um estorvo chamado selecao,o contra equador um primeiro tempo razoavel ...,24,83
110769,philippe coutinho cortado dunga convoca kaka e...,o tecnico dunga convocou noite deste domingo ...,3,83


In [184]:
data.query("cluster_wals_200 == 83").sample(10)

,title,text,date,category,subcategory,link,cluster_wals_200,cluster_wals_50
79913,selecao chega recife estrategia cbf apoio cont...,a selecao brasileira chegou recife nesta quint...,2016-03-24,esporte,None,http://www1.folha.uol.com.br/esporte/2016/03/1...,83,24
161146,sem provas cbf desiste denunciar suspeita raci...,a cbf confederacao brasileira futebol pretende...,2015-01-27,esporte,None,http://www1.folha.uol.com.br/esporte/2015/01/1...,83,24
78810,vacuo politico cbf dificulta mudanca comando s...,o futuro selecao brasileira atualmente da zona...,2016-03-31,esporte,None,http://www1.folha.uol.com.br/esporte/2016/03/1...,83,3
526,selecao empata fica chances superar recorde ar...,em partida pouca relevancia o futuro selecao b...,2017-05-10,esporte,None,http://www1.folha.uol.com.br/esporte/2017/10/1...,83,3
114009,somente tres convocados dunga disputaram elim...,dos jogadores convocados dunga os dois primei...,2015-09-18,esporte,None,http://www1.folha.uol.com.br/esporte/2015/09/1...,83,3
129728,com eliminacao neymar que cumprir suspensao el...,com eliminacao selecao brasileira o paraguai q...,2015-06-27,esporte,None,http://www1.folha.uol.com.br/esporte/2015/06/1...,83,3
66163,vai aparecer monte babaca falar diz neymar eli...,fora copa america centenario apos acordo o bar...,2016-06-13,esporte,None,http://www1.folha.uol.com.br/esporte/2016/06/1...,83,3
109997,apos derrota ricardo oliveira diz nao falta co...,a derrota selecao brasileira o chile santiag...,2015-08-10,esporte,None,http://www1.folha.uol.com.br/esporte/2015/10/1...,83,3
107446,cassio admite ansiedade frio barriga volta sel...,de volta a selecao brasileira goleiro cassio a...,2015-10-22,esporte,None,http://www1.folha.uol.com.br/esporte/2015/10/1...,83,3
147731,dunga faz misterio escalacao selecao brasileir...,o tecnico dunga evitou confirmar escalacao tim...,2015-03-25,esporte,None,http://www1.folha.uol.com.br/esporte/2015/03/1...,83,3


In [179]:
data.query("cluster_wals_200 == 114").sample(10)

,title,text,date,category,subcategory,link,cluster_wals_200,cluster_wals_50
77326,libertadores tem classificados oitavas nenhum...,nem argentinos brasileiros os dois paises mais...,2016-08-04,esporte,None,http://www1.folha.uol.com.br/esporte/2016/04/1...,114,40
35201,sorteio libertadores complica caminhos botafog...,nesta quartafeira conmebol definiu meio sorte...,2016-12-22,esporte,None,http://www1.folha.uol.com.br/esporte/2016/12/1...,114,24
42619,palmeiras vias acabar um jejum anos,o palmeiras em vias acabar um jejum anos camp...,2016-10-31,colunas,jucakfouri,http://www1.folha.uol.com.br/colunas/jucakfour...,114,24
144332,fase final estaduais classico minas no rio jan...,vasco flamengo enfrentam neste domingo h mara...,2015-12-04,esporte,None,http://www1.folha.uol.com.br/esporte/2015/04/1...,114,3
67868,sao paulo contrata meiaatacante selecao peruana,o sao paulo anunciou nesta quinta contratacao...,2016-02-06,esporte,None,http://www1.folha.uol.com.br/esporte/2016/06/1...,114,40
146315,boa fase argentino ate fe papa dao esperancas ...,no bairro classe media boedo buenos aires fila...,2015-01-04,esporte,None,http://www1.folha.uol.com.br/esporte/2015/04/1...,114,40
156258,inter sofre gols inicio estreia libertadores d...,o internacional resistiu m altitude la paz ca...,2015-02-18,esporte,None,http://www1.folha.uol.com.br/esporte/2015/02/1...,114,16
82557,morre roberto perfumo foi idolo zaga cruzeiro ...,o argentino roberto perfumo dos maiores jogado...,2016-10-03,esporte,None,http://www1.folha.uol.com.br/esporte/2016/03/1...,114,40
80674,apos empate bauza diz maior dificuldade a quan...,apos empate sao paulo o ituano itu neste dom...,2016-03-20,esporte,None,http://www1.folha.uol.com.br/esporte/2016/03/1...,114,40
3996,um terco clubes usou jogadores reservas durant...,o dilema tecnicos repete por uma temporada jog...,2017-08-09,esporte,None,http://www1.folha.uol.com.br/esporte/2017/09/1...,114,24


In [147]:
art.iloc[:,1:-1]

,cluster_words_weight,cluster_words_relevance,assunto_principal,assunto_secundario
0,cunha | camara | presidente | conselho | processo,cunha | cassacao | etica | pinato | camara,politica,cunha
1,sp | sao | paulo | brasil | folha,sp | leitorgrupofolhacombr | paulo | sao | lei...,midias,jornal
2,temer | presidente | joesley | jbs | ele,joesley | jbs | loures | batista | saud,politica,jbs
3,indice | china | economia | pontos | crescimento,indice | yuan | xangai | china | fed,economia,china
4,indigenas | governo | area | terras | segundo,indigenas | funai | indios | terras | desmatam...,indigenas,desmatamento
...,...,...,...,...
195,mins | hamilton | gp | massa | felipe,mins | vettel | hamilton | nasr | rosberg,esportes,automobilismo
196,ano | queda | trimestre | ibge | mes,ibge | trimestre | queda | desemprego | compar...,sociedade,desemprego
197,odebrecht | empresa | petrobras | lava | jato,leniencia | odebrecht | cgu | cartel | empreit...,lava jato,odebrecht
198,dia | paulo | personagem | vida | sempre,troya | klimpel | preparese | teles | seragusa,indefinido,indefinido


In [135]:
pd.DataFrame(a).join()

,cluster_wals_200
46,1481
177,1426
129,1160
83,1100
146,1093
...,...
21,1
37,1
53,1
44,1
